In [1]:
from sklearn import svm
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
iris = load_iris()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#open and read file

#file = open('DATA.csv')

#csvreader = csv.reader(file)
data= pd.read_csv("DATA.csv", delimiter=';')
df=pd.DataFrame(data)
df.head()

,STUDENT ID,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,COURSE ID,GRADE
0,STUDENT1,2,2,3,3,1,2,2,1,1,...,1,1,3,2,1,2,1,1,1,1
1,STUDENT2,2,2,3,3,1,2,2,1,1,...,1,1,3,2,3,2,2,3,1,1
2,STUDENT3,2,2,2,3,2,2,2,2,4,...,1,1,2,2,1,1,2,2,1,1
3,STUDENT4,1,1,1,3,1,2,1,2,1,...,1,2,3,2,2,1,3,2,1,1
4,STUDENT5,2,2,1,3,2,2,1,3,1,...,2,1,2,2,2,1,2,2,1,1


In [3]:
X = df.drop(['STUDENT ID', 'COURSE ID', 'GRADE'], axis = 1)
y = df['GRADE']

In [4]:
scaler = StandardScaler()
scaler.fit(X)

StandardScaler()

In [5]:
X_cols = df.columns.drop(['STUDENT ID', 'COURSE ID', 'GRADE'])
X = pd.DataFrame(scaler.transform(X), columns=X_cols) 

In [6]:
from imblearn.over_sampling import SMOTENC

sample = SMOTENC(categorical_features=[df.dtypes==object], random_state=42)

sampled_X, sampled_y = sample.fit_resample(X, y)

C:\Users\taile\Anaconda3\lib\site-packages\scipy\sparse\lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
C:\Users\taile\Anaconda3\lib\site-packages\scipy\sparse\lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
C:\Users\taile\Anaconda3\lib\site-packages\scipy\sparse\lil.py:518: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
sampled_X_train, sampled_X_test, sampled_y_train, sampled_y_test = train_test_split(sampled_X, sampled_y, test_size=0.3, random_state=42)

In [10]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

parameters = {'C':[0.001, 0.01, 0.1, 1, 5, 10], 'gamma':[0.001, .01, 0.1, 1]}
grid_search = GridSearchCV(SVC(kernel='rbf'), parameters, cv=10)

grid_search.fit(sampled_X_train, sampled_y_train)
oversampled_best_estimator = grid_search.best_estimator_

grid_search.fit(X_train, y_train)
regular_best_estimator = grid_search.best_estimator_

print('Regular Radial SVC: ' + oversampled_best_estimator.__str__())

C:\Users\taile\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  UserWarning,


Regular Radial SVC: SVC(C=5, gamma=0.1)


In [11]:
radial_clf = SVC(kernel='rbf', C=5, gamma=.1, random_state=42)
radial_clf.fit(sampled_X_train, sampled_y_train)

SVC(C=5, gamma=0.1, random_state=42)

In [13]:
from sklearn.metrics import confusion_matrix

prediction = radial_clf.predict(X_train)
confusion_matrix(y_train, prediction)

array([[ 3,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 25,  0,  0,  0,  0,  0,  0],
       [ 0,  4, 11,  0,  0,  0,  0,  0],
       [ 0,  3,  0, 12,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  6,  0,  0,  0],
       [ 0,  3,  0,  0,  0, 10,  1,  0],
       [ 0,  1,  0,  0,  0,  0,  7,  0],
       [ 0,  1,  0,  0,  0,  0,  0, 13]], dtype=int64)

In [14]:
from sklearn.metrics import accuracy_score

accuracy_score(y_train, prediction)

0.8613861386138614

In [15]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_train, prediction)

0.36633663366336633

In [17]:
prediction = radial_clf.predict(X_test)
confusion_matrix(y_test, prediction)

array([[ 5,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 10,  0,  0,  0,  0,  0,  0],
       [ 0,  4,  5,  0,  0,  0,  0,  0],
       [ 0,  2,  0,  4,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  3,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  3,  0,  0],
       [ 0,  2,  0,  0,  0,  0,  3,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  2]], dtype=int64)

In [18]:
accuracy_score(y_test, prediction)

0.7954545454545454

In [19]:
mean_absolute_error(y_test, prediction)

0.5454545454545454

In [21]:
from sklearn.model_selection import cross_val_score

cross_val_score(radial_clf, X_train, y_train, cv=10).mean()

C:\Users\taile\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  UserWarning,


0.24818181818181814

In [23]:
cross_val_score(radial_clf, X_test, y_test, cv=10).mean()

C:\Users\taile\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  UserWarning,


0.205